In [ ]:
import os, re
v_transformers = !pip show transformers
v_transformers = "\n".join(v_transformers)
v_transformers = re.search(r"version\:\s*([\d\.]+)", v_transformers.lower()).group(1).strip()
print("initial transformers version:", v_transformers)
if v_transformers != "4.20.1":
#     os.system('pip show transformers')
    os.system('pip uninstall -y transformers')
    os.system('pip uninstall -y tokenizers')
    os.system('python -m pip install --no-index --find-links=../input/pip-wheels-transformers-4-20-1/ transformers')
    os.system('python -m pip install --no-index --find-links=../input/pip-wheels-transformers-4-20-1/ tokenizers')
    import tokenizers
    import transformers
    print(f"tokenizers.__version__: {tokenizers.__version__}")
    print(f"transformers.__version__: {transformers.__version__}")
else:
    print("transformers is already up to date")

In [ ]:
import os, sys
sys.path.insert(0, "../input/kkillerfpzeff2ndplace/fpzeff/src")

In [ ]:
import pandas as pd, numpy as np
import torch

from tqdm.auto import tqdm
from pathlib import Path
from  datetime import datetime


import configs
import dataset
import inference

import gc

In [ ]:
IS_DEBUG = False

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# configs.TRAIN_ROOT = "../input/feedback-prize-effectiveness/train"
# configs.TRAIN_CSV_PATH = "../input/feedback-prize-effectiveness/train.csv"
# configs.SAMPLE_SUB_CSV_PATH = None

configs.TRAIN_ROOT = "../input/feedback-prize-effectiveness/test"
configs.TRAIN_CSV_PATH = "../input/feedback-prize-effectiveness/test.csv"
configs.SAMPLE_SUB_CSV_PATH = "../input/feedback-prize-effectiveness/sample_submission.csv"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(DEVICE)

# WEIGHTS

In [ ]:
import re
def get_fold(path):
    return int(re.search(r"_fold(\d+)_", str(path)).group(1))

In [ ]:
params = [
    
    inference.get_params(
        
        model_name="microsoft/deberta-v2-xlarge",
        use_mixup=False,
        forward_type="forward_full_oof",
        use_token_types=True,
        use_layer_norm=True,
        batch_size=4,
        maxlen=1792,
        num_workers=0,
        weight=.15,
        
        config_path="../input/kkillerfpzeff2ndplace/kkiller-transformers-data/microsoft_deberta-v2-xlarge-mnli",
        tokenizer_path="../input/kkillerfpzeff2ndplace/kkiller-transformers-data/microsoft_deberta-v2-xlarge-mnli",
        
        is_pickle=False,
        device=DEVICE,
        
        model_paths=[
            "../input/fpzeff-db2xl-psl-1span-notune-v3-1-m1152-weights/db2xl_psl_1span_v3_1_m1152/fpeff_microsoft_deberta-v2-xlarge_fold0_epoch_02_loss_v2_val_0.5704_20220820152247.pth",
            "../input/fpzeff-db2xl-psl-1span-notune-v3-1-m1152-weights/db2xl_psl_1span_v3_1_m1152/fpeff_microsoft_deberta-v2-xlarge_fold1_epoch_02_loss_v2_val_0.6005_20220820194712.pth",
        ],
        
        oof_name="oof_db2xl_fgp_psl_v3_1_1152_220823",
        
        dataset_module=dataset,
        inference_module=inference,
    ),
    
    
    inference.get_params(
        model_name="microsoft/deberta-xlarge",
        use_mixup=False,
        forward_type="forward_full_oof",
        use_token_types=True,
        use_layer_norm=True,
        batch_size=4,
        maxlen=1792,
        num_workers=0,
        weight=.25,
        config_path="../input/kkillerfpzeff2ndplace/kkiller-transformers-data/microsoft_deberta-xlarge",
        tokenizer_path="../input/kkillerfpzeff2ndplace/kkiller-transformers-data/microsoft_deberta-xlarge",
        is_pickle=False,
        device=DEVICE,
        model_paths=list(Path("../input/fpzeff-db1xl-psl-1span-notune-v3-1-m864-weights/db1xl_psl_1span_v3_1_m864/").glob("*.pth")),
        
        oof_name="oof_db1xl_fgp_psl_v3_1_864_220823",
        
        dataset_module=dataset,
        inference_module=inference,
    ),
    
    
    inference.get_params(
        model_name="microsoft/deberta-large",
        use_mixup=False,
        forward_type="forward_full_oof",
        use_token_types=True,
        use_layer_norm=True,
        batch_size=8,
        maxlen=1792,
        num_workers=0,
        weight=.25,
        config_path="../input/kkillerfpzeff2ndplace/kkiller-transformers-data/microsoft_deberta-large",
        tokenizer_path="../input/kkillerfpzeff2ndplace/kkiller-transformers-data/microsoft_deberta-large",
        is_pickle=False,
        device=DEVICE,
        model_paths=list(Path("../input/fpzeff-db1l-psl-1span-notune-v3-1-m1152-weights/db1l_psl_1span_v3_1_m1152/").glob("*.pth")),
        
        oof_name="oof_db1l_fgp_psl_v3_1_1152_220823",
        
        dataset_module=dataset,
        inference_module=inference,
    ),
    
    
    inference.get_params(
        model_name="microsoft/deberta-v3-large",
        use_mixup=False,
        forward_type="forward_full_oof",
        use_token_types=True,
        use_layer_norm=True,
        batch_size=8,
        maxlen=1792,
        num_workers=0,
        weight=.35,
        config_path="../input/kkillerfpzeff2ndplace/kkiller-transformers-data/microsoft_deberta-v3-large",
        tokenizer_path="../input/kkillerfpzeff2ndplace/kkiller-transformers-data/microsoft_deberta-v3-large",
        is_pickle=False,
        device=DEVICE,
        model_paths=list(Path("../input/fpzeff-db3l-psl-1span-notune-v3-1-m1152-weights/db3l_psl_1span_v3_1_m1152/").glob("*.pth")),
        
        oof_name="oof_full_db3l_fgp_psl_v3_1_1152_220822",
        
        dataset_module=dataset,
        inference_module=inference,
    ),
    
]

S = sum([param["weight"] for param in params])
assert abs(S- 1.0) < 1e-3, S

print("TOTAL UNIQUE MODELS:", len(params))
print("TOTAL  CHECKPOINTS:", sum([len(param["model_paths"]) for param in params]))
print("MODELS PER CHKPT;", [len(param["model_paths"]) for param in params])

In [ ]:
df = dataset.read_train_csv(configs.TRAIN_CSV_PATH, nrows=None, is_test=True)
df["train_root"] = configs.TRAIN_ROOT

train_roots = dict(zip(df["id"], df["train_root"]))  
essays = {uuid: dataset.read_from_id(uuid, root=train_roots[uuid]) for uuid in tqdm(df["id"].unique(), desc="reading essays")}

df["nchars"] = df["id"].map({uuid: len(essays[uuid].split()) for uuid in tqdm(df["id"].unique(), desc="nchars")})
df["is_true_obs"] = True

uuids = dict(zip(df["id"], -df["nchars"]))
uuids = sorted(list(uuids.keys()), key=lambda uuid: uuids[uuid])

if IS_DEBUG:
    uuids = uuids[:100]
    df = df[df["id"].isin(uuids)].reset_index(drop=True)

print("nuuids", len(uuids))

print(f"len(uuids): {len(uuids)}")

print(df.shape)
df[df["id"].isin(uuids[:3])]

In [ ]:
get_fold(params[0]["model_paths"][0])

In [ ]:
def expand_oof(full_oof):
    full_oof_list = []
    for (essay_id, discourse_ids, preds) in full_oof["preds"]:

        temp = {}
        for discourse_id, pred in zip(discourse_ids, preds):
            if discourse_id in temp:
                temp[discourse_id].append(pred)
            else:
                temp[discourse_id] = [pred]


        for discourse_id, pred in temp.items():
            pred = np.stack(pred)

            full_oof_list.append(
                {
                    "essay_id": essay_id,
                    "discourse_id": discourse_id,
                    "preds": pred,
                    "fold": full_oof["info"]["fold"],
                }

            )

    full_oof = pd.DataFrame(full_oof_list)
        
    return full_oof

In [ ]:
def predict_for_param(param):

    cols = ["Ineffective", "Adequate", "Effective"]

    all_preds = {}
    
    model_paths = {get_fold(path): path for path in param["model_paths"]}
    inference_module = param.get("inference_module", inference)
    
    model_name = Path(param["model_paths"][0]).parent.stem
    
    print(f"model: {model_name}  num_folds: {len(model_paths)}")
    
    folds_bar = tqdm(sorted(model_paths), desc=model_name)
    
    data = inference_module.load_data(
        uuids=uuids,
        param=param,
        df=df,
        essays=essays,
        bar_for_data=True,
    )
    
    for fold in folds_bar:
        model_path = Path(model_paths[fold])
        
        print(f"model: {model_name}  fold: {fold}")
        
        ckpt = model_path.stem
        
        param["model_paths"] = [model_path]
        
        preds  = inference_module.predict_from_param(
            uuids=uuids,
            param=param,
            data=data,
            df=df,
            essays=essays,
#             bar_for_models=IS_DEBUG,
            bar_for_models=True,
    #         bar_for_data=IS_DEBUG,
            bar_for_data=False,
            bar=False,
            do_full_oof=True,
        )
        
        res = {
            "preds": preds,
            "info": {
                "fold": fold,
                "weight": param["weight"],
                "fold_name": configs.FOLD_COL_NAME,
                "model_name": model_name,
                "ckpt": ckpt,
            }
        }
        
        all_preds[fold] = expand_oof(res)
        
        del res
        gc.collect()
        torch.cuda.empty_cache()
    
    del data
    gc.collect()
    torch.cuda.empty_cache()
        
    return all_preds

In [ ]:
def sorted_quantile(array, q):
    array = np.array(array)
    n = len(array)
    index = (n - 1) * q
    left = np.floor(index).astype(int)
    fraction = index - left
    right = left
    right = right + (fraction > 0).astype(int)
    i, j = array[left], array[right]
    return i + (j - i) * fraction

from scipy.stats import entropy
#make features
def get_xgb_features(train_df,prob_sequences):
    features2calculate=[f"instability_{i}" for i in range(4)]+\
    [f"begin_{i}" for i in range(3)]+\
    [f"end_{i}" for i in range(3)]#+\
    #["entropy"]

    calculated_features=[]
    for i,prob_seq in tqdm(enumerate(prob_sequences)):

        tmp=[]
        #quants = np.linspace(0,1,n_quan)
        prob_seq=np.array(prob_seq)
        instability = []
        #all_quants=[] 
        tmp.append(np.diff(prob_seq[:,:],0).mean(0))
        tmp.append([(np.diff(prob_seq[:,[1,2]].sum(1))**2).mean()])

        tmp.append(prob_seq[:5,:].mean(0))
        tmp.append(prob_seq[-5:,:].mean(0))

        calculated_features.append(np.concatenate(tmp))


    train_df[features2calculate]=calculated_features
    train_df['len']=[len(s) for s in prob_sequences]

    calculated_features=np.array(calculated_features)
    calculated_features.shape

    p_features=[]
    n_features=[]
    neighbor_features=['Ineffective','Adequate','Effective','discourse_type']
    neighbor_features_values=train_df[neighbor_features].values
    for i in tqdm(range(len(train_df))):
        if i>1 and train_df['essay_id'].iloc[i]==train_df['essay_id'].iloc[i-1]:
            p_features.append(neighbor_features_values[i-1])
        else:
            p_features.append(neighbor_features_values[i])

        if i<(len(train_df)-1) and train_df['essay_id'].iloc[i]==train_df['essay_id'].iloc[i+1]:
            n_features.append(neighbor_features_values[i+1])
        else:
            n_features.append(neighbor_features_values[i])

    train_df[[f+"_previous" for f in neighbor_features]]=p_features
    train_df[[f+"_next" for f in neighbor_features]]=n_features

    train_df['mean_Ineffective']=train_df.groupby("essay_id")["Ineffective"].transform("mean")
    train_df['mean_Adequate']=train_df.groupby("essay_id")["Adequate"].transform("mean")
    train_df['mean_Effective']=train_df.groupby("essay_id")["Effective"].transform("mean")

    train_df['std_Ineffective']=train_df.groupby("essay_id")["Ineffective"].transform("std")
    train_df['std_Adequate']=train_df.groupby("essay_id")["Adequate"].transform("std")
    train_df['std_Effective']=train_df.groupby("essay_id")["Effective"].transform("std")

    train_df['discourse_count']=train_df.groupby("essay_id")['discourse_type'].transform("count")

    cnts=train_df.groupby('essay_id')['discourse_type'].apply(lambda x: x.value_counts())

    #new_df=[]
    discourse_types=['Claim','Evidence','Concluding Statement','Lead','Position','Counterclaim','Rebuttal']
    value_count_hash={}
    for t in discourse_types:
        value_count_hash[t]={}
    for key in cnts.keys():
        value_count_hash[key[1]][key[0]]=cnts[key]

    discourse_cnts=[]    
    for essay_id in train_df['essay_id'].unique():
        row=[essay_id]
        for d in discourse_types:
            try:
                row.append(value_count_hash[d][essay_id])
            except:
                row.append(0)
        discourse_cnts.append(row)

    discourse_cnts=pd.DataFrame(discourse_cnts,columns=['essay_id']+[f'{d}_count' for d in discourse_types])    
    #discourse_cnts

    train_df=train_df.merge(discourse_cnts,how='left',on='essay_id')
    train_df

    #train_df

    return train_df

neighbor_features=['Ineffective','Adequate','Effective','discourse_type']
discourse_types=['Claim','Evidence','Concluding Statement','Lead','Position','Counterclaim','Rebuttal']

features=["Ineffective","Adequate","Effective",
          "instability_0","instability_1","instability_2","instability_3",
          "len","discourse_type"]
features+=[f"begin_{i}" for i in range(3)]
features+=[f"end_{i}" for i in range(3)]

features=features+[f+"_previous" for f in neighbor_features]+[f+"_next" for f in neighbor_features]+\
['mean_Ineffective','mean_Adequate','mean_Effective']+['std_Ineffective','std_Adequate','std_Effective']+\
['discourse_count']+[f'{d}_count' for d in discourse_types]

In [ ]:
import xgboost as xgb
subs=[]
weights=[]
for param in tqdm(params):
    all_preds = predict_for_param(param)
    weights.append(param['weight'])
    N_XGB_FOLDS=6
    TOKEN_START_POS=2
    #XGB_PATH="../input/kkiller-stacking-final/kkiller_stacking/oof_db1l_fgp_psl_v3_1_1152_220823/gbm_models"
    XGB_PATH=f"../input/kkiller-stacking-final/kkiller_stacking/{param['oof_name']}/gbm_models"
    label_mapping={'Ineffective': 0, 'Adequate': 1, 'Effective': 2}
    test=pd.read_csv("../input/feedback-prize-effectiveness/test.csv")
    xgb_preds=[]
    
    
    for key in all_preds.keys():
    #for key in [3]:
        pred_df=all_preds[key]
        sub=pd.DataFrame(columns=['discourse_id']+list(label_mapping.keys()))
        sub['discourse_id']=pred_df['discourse_id']
        
        preds=[s[1] for s in pred_df['preds']]
        sub[list(label_mapping.keys())]=preds
        sub=sub.merge(test[['discourse_id','discourse_type','essay_id']],how='left',on='discourse_id')
        prob_sequences=[s[TOKEN_START_POS:] for s in pred_df['preds']]
        sub=get_xgb_features(sub,prob_sequences)

        for f in features:
                if f not in ['discourse_type_previous','discourse_type_next','discourse_type']:
                    sub[f]= sub[f].astype('float')
                else:    
                    sub[f]= sub[f].astype('category')

        d_test = xgb.DMatrix(sub[features],enable_categorical=True)

        for xgb_fold in range(N_XGB_FOLDS):
            #xgb_model_loaded = pickle.load(open(f"{exp.XGB_PATH}/xgb_{xgb_fold}.p", "rb"))
            xgb_model_loaded = xgb.Booster()
            xgb_model_loaded.load_model(f"{XGB_PATH}/xgb_{xgb_fold}.json")
            xgb_preds.append(xgb_model_loaded.predict(d_test))
    xgb_preds=np.stack(xgb_preds)
    print(xgb_preds.shape)
    xgb_preds=xgb_preds.mean(0)


    submission=pd.read_csv("../input/feedback-prize-effectiveness/sample_submission.csv")

    discourse_ids=list(submission['discourse_id'])
    for i in range(len(sub)):
        index=discourse_ids.index(sub['discourse_id'].iloc[i])
        submission["Ineffective"].iloc[index]=xgb_preds[i,0]
        submission["Adequate"].iloc[index]=xgb_preds[i,1]
        submission["Effective"].iloc[index]=xgb_preds[i,2]

    #submission.to_csv("submission.csv",index=False)
    subs.append(submission)
    print(submission)
    # Call Stacker here

In [ ]:
# import os
# for param in tqdm(params):
#     #print(param['oof_name'])
#     XGB_PATH=f"../input/kkiller-stacking-final/kkiller_stacking/{param['oof_name']}/gbm_models"
#     os.system(f"ls {XGB_PATH}")
#     #os.system()



In [ ]:
assert len(subs)==len(weights)

print(weights)

weights=np.array(weights)
weights=weights/weights.sum()
submission=subs[0].copy()
submission["Ineffective"]=submission["Ineffective"].values*weights[0]
submission["Adequate"]=submission["Adequate"].values*weights[0]
submission["Effective"]=submission["Effective"].values*weights[0]


for sub,weight in zip(subs[1:],weights[1:]):
    submission["Ineffective"]=submission["Ineffective"].values+sub["Ineffective"].values*weight
    submission["Adequate"]=submission["Adequate"].values+sub["Adequate"].values*weight
    submission["Effective"]=submission["Effective"].values+sub["Effective"].values*weight

submission.to_csv("submission.csv",index=False)